<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/TE_WORD2VEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load DF + preprocessing

In [ ]:
# Declare whether you are on Colab or local
colab = True

In [ ]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')


  #set path to data in Google Drive
  data_path = "/content/drive/MyDrive/Colab Notebooks/TWSM Project/2022_Analytics Lab Student Projects/Data"


  #install required packages
  #!pip install duckdb
  #!pip install -U sentence-transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import pickle

In [ ]:
# Load new noun_phrases from sentences.csv data and ontology
if colab==True:
  noun_phrases = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TWSM Project/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/SynDet/SynDet_entities_train.csv")
else:
  noun_phrases = pd.read_csv("../SynDet_entities_train.csv")


if colab==True:
  ontology_syn = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TWSM Project/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/SynDet/SynDet_ontology_train.csv")
else:
  ontology_syn = pd.read_csv("../SynDet_ontology_train.csv")

In [ ]:
noun_phrases.head(2)

,Unnamed: 0,article_id,sentence_id,ent_id,noun_phrases,true_ent_id
0,0,188,188_3753_3772,videotex,which,0
1,1,188,188_3753_3772,videotex,we,0


In [ ]:
ontology_syn.head(2)

,Unnamed: 0,entity_id,category,label,synonym
0,0,sampling,methodological entity,SAMPLING,sampling
1,0,data analysis method,methodological entity,ANALYSIS_METHOD,data analysis investigation


In [ ]:
#convert column to strings from noun_phrases
noun_phrases['noun_phrases'] = noun_phrases['noun_phrases'].astype(str)

# filter df noun_phrases for article 188
noun_phrases = noun_phrases.loc[noun_phrases["article_id"]==188]

# Remove Stopwords? 

In [163]:
#tokenizing the noun_phrases
from nltk.tokenize import RegexpTokenizer

tokenizer_reg = RegexpTokenizer(r'\w+')

noun_phrases["tokens"] = noun_phrases["noun_phrases"].apply(tokenizer_reg.tokenize)
noun_phrases.head()

,Unnamed: 0,Unnamed: 0.1,article_id,sentence_id,ent_id,noun_phrases,true_ent_id,tokens,similarity_score,ontology_txt
0,0,0,188,188_3753_3772,videotex,which,0,[which],0.4732733882113142,IS/IT in healthcares
1,1,1,188,188_3753_3772,videotex,we,0,[we],0.5631700901822527,interorganizational I
2,2,2,188,188_3753_3772,videotex,an information,0,"[an, information]",0.6066101391194407,medical information
3,3,3,188,188_3753_3772,videotex,communication resource,0,"[communication, resource]",0.6281378354394097,IS/IT resource management
4,4,4,188,188_3753_3772,videotex,the Minitel system,0,"[the, Minitel, system]",0.5728775449540222,system in governments


#### Downsampling the synonyms 

In [ ]:
# Check number of synonyms per entity_id
ent_sy_group = ontology_syn.groupby("entity_id")[["synonym"]].nunique().sort_values(by="synonym", ascending=False)

In [ ]:
# Downsample entities with more than 100 synonyms to a 100 synoyms (but always include "main entity_id" in subsample)
idx = ent_sy_group[ent_sy_group["synonym"]>100].index.tolist()

dfs = []
for entity in idx:
  t1 = ontology_syn[(ontology_syn.entity_id==entity)&(ontology_syn.synonym==entity)]
  t2 = ontology_syn[(ontology_syn.entity_id==entity)&(ontology_syn.synonym!=entity)].sample(99)
  dfs.append(pd.concat([t1,t2]))
dfs.append(ontology_syn.loc[~ontology_syn["entity_id"].isin(idx)]) # Append dataframe for all entities with less then 100 synonyms to list

# Combine all dataframes 
ontology_syn_ds = pd.concat(dfs).reset_index(drop=True)

In [ ]:
#List of unique entities from ontology with synonyms with downsampling
ontology_syn_ds_unique = ontology_syn_ds.drop_duplicates(subset="synonym")
ontology_syn_ds_unique.head()

,Unnamed: 0,entity_id,category,label,synonym
0,4326,individual participant,study object,PARTICIPANTS,individual participant
1,125082,individual participant,study object,PARTICIPANTS,senior junior citizens ( )S
2,198526,individual participant,study object,PARTICIPANTS,junior and knowledge security CIOss
3,191036,individual participant,study object,PARTICIPANTS,senior and knowledge executive CIOss
4,80692,individual participant,study object,PARTICIPANTS,professional usable professionals ( )


In [ ]:
# Create Dictionary noun_phrases
from gensim.corpora import Dictionary
dictionary_noun_phrases=Dictionary(noun_phrases.tokens)
print(dictionary_noun_phrases)

Dictionary(4943 unique tokens: ['CITE_b97', 'END_CITE', 'changes', 'driven', 'technology']...)


In [ ]:
#tokenizing the ontology_syn_ds
ontology_syn_ds_unique["tokens"] = ontology_syn_ds_unique.loc[:,"synonym"].apply(tokenizer_reg.tokenize)
ontology_syn_ds_unique.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,entity_id,category,label,synonym,tokens
0,4326,individual participant,study object,PARTICIPANTS,individual participant,"[individual, participant]"
1,127759,individual participant,study object,PARTICIPANTS,IT and executive IS member,"[IT, and, executive, IS, member]"
2,123374,individual participant,study object,PARTICIPANTS,IS and junior chief CIOss,"[IS, and, junior, chief, CIOss]"
3,200527,individual participant,study object,PARTICIPANTS,executive and business usable users,"[executive, and, business, usable, users]"
4,171829,individual participant,study object,PARTICIPANTS,senior and business junior CIOs,"[senior, and, business, junior, CIOs]"


## Gensim - Word2Vec 

In [ ]:
#load model as api
#import gensim.downloader as api
#model = api.load('word2vec-google-news-300')

In [ ]:
#download model
#import gensim.downloader
#word2vec = gensim.downloader.load("word2vec-google-news-300")

In [ ]:
#import model from drive
from gensim.models import KeyedVectors
# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/TWSM Project/GoogleNews-vectors-negative300.bin.gz', binary=True)

### Word Mover’s Distance

Word Movers Distance: The distance between the two documents is the minimum cumulative distance that all words in document 1 need to travel to exactly match document 2

In [165]:
# convert ontology_syn_ds_unique_list to a list of tokens
ontology_syn_ds_unique_list = ontology_syn_ds_unique["synonym"].apply(lambda x:x.split()).to_list()

In [ ]:
#import Word Movers Distance Similarity and apply on the word2vec model and the ontology_syn_ds_unique_list
#search only for the best match: num_best = 1
from gensim.similarities import WmdSimilarity
index = WmdSimilarity(ontology_syn_ds_unique_list, model.wv, num_best = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [171]:
#try the WmdSimilarity on one example in the noun phrase table
query = noun_phrases["tokens"].iloc[10]
sims = index[query]  #look-up in the WmdSimilarity index
print(sims)

[(32304, 0.5970549674148199)]


In [175]:
#check the match found by the WmdSimilarity
print("noun phrase: "+str(query))
print("matched entity: "+str(ontology_syn_ds_unique_list[32304]))
print("score: "+ str(sims[0][1]))

noun phrase: ['further', 'ampli', 'ed']
matched entity: ['further', 'research']
score: 0.5970549674148199


In [ ]:
#finding the best synonym for noun_phrases (already filtered to: article_id==188) and get the similarity_score (WMD-score)

similarity_score = []
ontology_txt = []

num_best=1
for x in noun_phrases.tokens:
  query = x
  sims = index[x]
  for i in range(num_best):
    try:
      similarity_score.append(sims[i][1])
      ontology_txt.append(ontology_syn_ds_unique_list[sims[i][0]])
    except IndexError:
        similarity_score.append("not in index range")
        ontology_txt.append("not in index range")

In [189]:
#add to df noun_phrases
noun_phrases_scores = noun_phrases
noun_phrases["similarity_score"] = similarity_score
noun_phrases["ontology_txt"] = ontology_txt

In [190]:
noun_phrases_scores.head(20)

,Unnamed: 0,Unnamed: 0.1,article_id,sentence_id,ent_id,noun_phrases,true_ent_id,tokens,similarity_score,ontology_txt
0,0,0,188,188_3753_3772,videotex,which,0,[which],0.473273,"[IS/IT, in, healthcares]"
1,1,1,188,188_3753_3772,videotex,we,0,[we],0.56317,"[interorganizational, I]"
2,2,2,188,188_3753_3772,videotex,an information,0,"[an, information]",0.60661,"[medical, information]"
3,3,3,188,188_3753_3772,videotex,communication resource,0,"[communication, resource]",0.628138,"[IS/IT, resource, management]"
4,4,4,188,188_3753_3772,videotex,the Minitel system,0,"[the, Minitel, system]",0.572878,"[system, in, governments]"
5,5,5,188,188_3828_3848,competitive advantage,It,0,[It],0.481062,"[think, alouds]"
6,6,6,188,188_3828_3848,competitive advantage,any rm,0,"[any, rm]",0.487762,"[does, not, have, any, significant, effects]"
7,7,7,188,188_3828_3848,competitive advantage,the eld,0,"[the, eld]",0.587938,"[of, the, papers]"
8,8,8,188,188_3828_3848,competitive advantage,so competitive advantages,0,"[so, competitive, advantages]",0.689987,"[competitive, advantages]"
9,9,9,188,188_3848_3883,government,This,0,[This],0.463592,"[think, alouds]"


In [199]:
#save df
#noun_phrases_scores.to_csv("/content/drive/MyDrive/Colab Notebooks/TWSM Project/noun_phrases_scores.csv")  

In [200]:
#noun_phrases_scores = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TWSM Project/noun_phrases_scores.csv")

In [230]:
noun_phrases_scores.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,article_id,sentence_id,ent_id,noun_phrases,true_ent_id,tokens,similarity_score,ontology_txt
0,0,0,0,0,188,188_3753_3772,videotex,which,0,['which'],0.4732733882113142,"['IS/IT', 'in', 'healthcares']"
1,1,1,1,1,188,188_3753_3772,videotex,we,0,['we'],0.5631700901822527,"['interorganizational', 'I']"
2,2,2,2,2,188,188_3753_3772,videotex,an information,0,"['an', 'information']",0.6066101391194407,"['medical', 'information']"
3,3,3,3,3,188,188_3753_3772,videotex,communication resource,0,"['communication', 'resource']",0.6281378354394097,"['IS/IT', 'resource', 'management']"
4,4,4,4,4,188,188_3753_3772,videotex,the Minitel system,0,"['the', 'Minitel', 'system']",0.5728775449540222,"['system', 'in', 'governments']"


In [49]:
#cleaning the column "ontology_txt"

In [231]:
noun_phrases_scores["ontology_txt"] = noun_phrases_scores["ontology_txt"].str.replace(",", "")

In [232]:
noun_phrases_scores["ontology_txt"] = noun_phrases_scores["ontology_txt"].str.replace("'", "")

In [233]:
noun_phrases_scores["ontology_txt"] = noun_phrases_scores["ontology_txt"].str.replace("[", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [234]:
noun_phrases_scores["ontology_txt"] = noun_phrases_scores["ontology_txt"].str.replace("]", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [235]:
noun_phrases_scores.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,article_id,sentence_id,ent_id,noun_phrases,true_ent_id,tokens,similarity_score,ontology_txt
0,0,0,0,0,188,188_3753_3772,videotex,which,0,['which'],0.4732733882113142,IS/IT in healthcares
1,1,1,1,1,188,188_3753_3772,videotex,we,0,['we'],0.5631700901822527,interorganizational I
2,2,2,2,2,188,188_3753_3772,videotex,an information,0,"['an', 'information']",0.6066101391194407,medical information
3,3,3,3,3,188,188_3753_3772,videotex,communication resource,0,"['communication', 'resource']",0.6281378354394097,IS/IT resource management
4,4,4,4,4,188,188_3753_3772,videotex,the Minitel system,0,"['the', 'Minitel', 'system']",0.5728775449540222,system in governments


In [236]:
#drop not required columns
noun_phrases_clean = noun_phrases_scores.drop(columns=["article_id", "Unnamed: 0.1", "Unnamed: 0", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1", "tokens", "true_ent_id"])
noun_phrases_clean

,sentence_id,ent_id,noun_phrases,similarity_score,ontology_txt
0,188_3753_3772,videotex,which,0.4732733882113142,IS/IT in healthcares
1,188_3753_3772,videotex,we,0.5631700901822527,interorganizational I
2,188_3753_3772,videotex,an information,0.6066101391194407,medical information
3,188_3753_3772,videotex,communication resource,0.6281378354394097,IS/IT resource management
4,188_3753_3772,videotex,the Minitel system,0.5728775449540222,system in governments
...,...,...,...,...,...
3222,188_3675_3704,government,mind,0.49290139466647426,thinking alouds
3223,188_3704_3727,government,the French people,0.5347515368856867,statistic the hypothesis
3224,188_3704_3727,government,the government,0.6226555134885214,government I
3225,188_3704_3727,government,the French superhighway,0.529844106122733,theory of the commons


In [208]:
#save clean df
#noun_phrases_clean.to_csv("/content/drive/MyDrive/Colab Notebooks/TWSM Project/wmd_syn_clean.csv")  

In [237]:
#merge dfs to get main_entity_id (=ent_id)
noun_phrases_mapped = noun_phrases_clean.merge(ontology_syn_ds_unique, 
                                               how="left", left_on="ontology_txt", right_on="synonym").drop(columns=["category", "label", "synonym"])

In [238]:
#drop not required columns and rename columns
sim_scores_sample = noun_phrases_mapped.drop(columns=["ontology_txt", "Unnamed: 0"]).rename(columns={"noun_phrases":"noun_phrase_txt", "similarity_score":"score", "entity_id": "main_entity_id", "ent_id": "entity_id"})

In [239]:
sim_scores_sample = sim_scores_sample[["score", "sentence_id", "entity_id", "noun_phrase_txt", "main_entity_id"]]
sim_scores_sample.head()

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id
0,0.4732733882113142,188_3753_3772,videotex,which,health information system
1,0.5631700901822527,188_3753_3772,videotex,we,interorganizational system
2,0.6066101391194407,188_3753_3772,videotex,an information,healthcare data
3,0.6281378354394097,188_3753_3772,videotex,communication resource,IT resource management
4,0.5728775449540222,188_3753_3772,videotex,the Minitel system,government system


In [240]:
entities_sample = noun_phrases.loc[noun_phrases["article_id"]==188]

In [241]:
# Merge sim scores on entities sample to get "true" entity id
sim_scores_sample_mapped = sim_scores_sample.merge(entities_sample
                                              [["sentence_id", "noun_phrases", "true_ent_id"]],
                                               how="left", left_on=["sentence_id", "noun_phrase_txt"], 
                                               right_on=["sentence_id", "noun_phrases"])\
                                               .drop(columns=["noun_phrases"])

In [242]:
# Define columns which should be checked for duplicates
col_dupl = sim_scores_sample_mapped.columns[1:].to_list()

#Drop duplicates
sim_scores_sample_mapped = sim_scores_sample_mapped.drop_duplicates(subset=col_dupl)

In [243]:
#Merge sim scores again on entities sample to get "true" entity id, but this time with the true entity id which was found by Roland
sim_scores_sample_mapped_f =sim_scores_sample_mapped.merge(entities_sample[["sentence_id", "ent_id"]], 
                                                           how="left", left_on=["sentence_id", "main_entity_id"], 
                                                           right_on=["sentence_id", "ent_id"])\
                                                           .rename(columns={"ent_id":"true_ent_id_cmpl_sen"})\
                                                           .drop_duplicates()

In [244]:
# Define function to combine both columns "true_ent_id" & "true_ent_id_cmpl_sen"
def true_ent_label(x):

  if x.true_ent_id!="0":
    return x.true_ent_id
  else:
    return x.true_ent_id_cmpl_sen

In [245]:
# Compare both columns and store correct value  in column true_entity_id
sim_scores_sample_mapped_f["true_entity_id"] = sim_scores_sample_mapped_f.apply(lambda x: true_ent_label(x), 
                                                                                axis=1, result_type="expand")

#Fill all NaN Values with 0 (no match)
sim_scores_sample_mapped_f["true_entity_id"] = sim_scores_sample_mapped_f["true_entity_id"].fillna(0)

#Drop columns "true_ent_id" & "true_ent_id_cmpl_sen"
sim_scores_sample_mapped_f = sim_scores_sample_mapped_f.drop(columns=["true_ent_id", "true_ent_id_cmpl_sen"])

In [246]:
sim_scores_sample_mapped_f.head(3)


,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id,true_entity_id
0,0.4732733882113142,188_3753_3772,videotex,which,health information system,0
1,0.5631700901822527,188_3753_3772,videotex,we,interorganizational system,0
2,0.6066101391194407,188_3753_3772,videotex,an information,healthcare data,0


In [247]:
# Check wheter main_entity based on score is identical to "true" main entity label 
sim_scores_sample_mapped_f["correct_match"] = sim_scores_sample_mapped_f.main_entity_id==sim_scores_sample_mapped_f.true_entity_id

In [248]:
# Check results for one sample sentence
sim_scores_sample_mapped_f[(sim_scores_sample_mapped_f.sentence_id=="188_5984_6028")].sort_values(by="score", ascending=False)#&(sim_scores_samp

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id,true_entity_id,correct_match
1204,1.0,188_5984_6028,information superhighway,information infrastructure,information technology infrastructure,information technology infrastructure,True
1255,1.0,188_5984_6028,private sector,information infrastructure,information technology infrastructure,information technology infrastructure,True
1306,1.0,188_5984_6028,hardware,information infrastructure,information technology infrastructure,information technology infrastructure,True
1317,1.0,188_5984_6028,hardware,application,IT workforce,0,False
1215,1.0,188_5984_6028,information superhighway,application,IT workforce,0,False
1266,1.0,188_5984_6028,private sector,application,IT workforce,0,False
1357,1.0,188_5984_6028,information technology infrastructure,information infrastructure,information technology infrastructure,information technology infrastructure,True
1368,1.0,188_5984_6028,information technology infrastructure,application,IT workforce,0,False
1318,0.6956238842940197,188_5984_6028,hardware,service development,information services development,0,False
1216,0.6956238842940197,188_5984_6028,information superhighway,service development,information services development,0,False


In [249]:
#remove the values where loop did not get an index ("not in index range")
sim_scores_sample_mapped_f1 = sim_scores_sample_mapped_f[~sim_scores_sample_mapped_f.score.str.contains("not in index range")]

In [250]:
#convert str to float
sim_scores_sample_mapped_f1["score"] = sim_scores_sample_mapped_f1["score"].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [251]:
# Create bins of similarity score to check the number of correct matches compared to the labels of the database
grouped_df = sim_scores_sample_mapped_f1.groupby(pd.cut(sim_scores_sample_mapped_f1.score, np.arange(0.,1.1,0.1)))[["correct_match"]]
corr_match_bins = np.round(grouped_df.sum()/grouped_df.count(),2).rename(columns={"correct_match":"pct_correct_in_bin"})
corr_match_bins["no_correct"]= grouped_df.sum()
corr_match_bins["total"]= grouped_df.count()
corr_match_bins = corr_match_bins[["total", "no_correct", "pct_correct_in_bin"]]
corr_match_bins = corr_match_bins.iloc[::-1] #reverse order

In [252]:
corr_match_bins

,total,no_correct,pct_correct_in_bin
score,,,
"(0.9, 1.0]",227,56,0.25
"(0.8, 0.9]",0,0,NaN
"(0.7, 0.8]",110,27,0.25
"(0.6, 0.7]",834,188,0.23
"(0.5, 0.6]",871,65,0.07
"(0.4, 0.5]",563,15,0.03
"(0.3, 0.4]",0,0,NaN
"(0.2, 0.3]",0,0,NaN
"(0.1, 0.2]",0,0,NaN


In [253]:
#pre pro - remove stop words
#define WMD Score (what is it exactly - compareable to CS?)